In [1]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models import Word2Vec

导入库函数

In [2]:
train_filename = 'E:/tulip-flip/flip01/data/train.json'
test_filename = 'E:/tulip-flip/flip01/data//test.json'

In [3]:
train = pd.read_json(train_filename)

In [4]:
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
label = list(train['cuisine'].unique())

In [6]:
def label_dataset(row):
    num_label = label.index(row)
    return num_label

train['label'] = train['cuisine'].apply(label_dataset)

In [7]:
train.tail()

,cuisine,id,ingredients,label
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ...",16
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b...",6
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte...",16
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli...",8
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground...",7


In [8]:
stop = ['，','。','[',']','{','}','"',':',',']

In [9]:
train['ingredients'] = [[j for j in i if j not in stop]for i in train['ingredients']]

In [10]:
train.tail()

,cuisine,id,ingredients,label
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ...",16
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b...",6
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte...",16
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli...",8
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground...",7


In [11]:
sentences = train['ingredients']

In [12]:
model = Word2Vec(sentences,sg=1,size=100,window=5,min_count=1,negative=3,sample=0.001,hs=1,workers=4)

In [13]:
model.wv.save_word2vec_format("word2vec",binary = "Ture")  #通过该方式保存的模型，能通过文本格式打开，也能通过设置binary是否保存为二进制文件。但该模型在保存时丢弃了树的保存形式（详情参加word2vec构建过程，以类似哈夫曼树的形式保存词），所以在后续不能对模型进行追加训练

In [14]:
model

In [15]:
model.most_similar("green chile",topn=10)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[("Campbell's Condensed Cream of Chicken Soup", 0.7645553350448608),
 ('yellow hominy', 0.7600508332252502),
 ('mild green chiles', 0.7526130676269531),
 ('green chile sauce', 0.7497570514678955),
 ('picante sauce', 0.7441935539245605),
 ('reduced fat monterey jack cheese', 0.7440881133079529),
 ('fat-free cheddar cheese', 0.743409276008606),
 ('40% less sodium taco seasoning mix', 0.7433062791824341),
 ('mexican style 4 cheese blend', 0.7430845499038696),
 ('40% less sodium taco seasoning', 0.7428058385848999)]

In [16]:
type(model)

gensim.models.word2vec.Word2Vec

In [17]:
def getVector_v2(sentences, model):
        vector_list = [ word2vec_model[k] for k in sentences if k in model]
        vector_df = pd.DataFrame(vector_list)
        sentences_vector = vector_df.mean(axis=0).values
        return sentences_vector

In [18]:
import time

In [19]:
print(model)

Word2Vec(vocab=6714, size=100, alpha=0.025)


In [20]:
print(model['green chile'])

[-0.02592937 -0.601608   -0.07893993  0.23143396 -0.4421613  -0.02062472
  0.24397255 -0.17655072 -0.33756706 -0.05419874  0.42510745 -0.04337338
 -0.10163632 -0.1105748  -0.26092902 -0.051609   -0.23915659 -0.49156788
  0.31080538 -0.35797817  0.47740215 -0.02815145 -0.31961927 -0.3370641
  0.1079415   0.1966532   0.03009647 -0.13004082  0.44800925 -0.08126725
 -0.06677918 -0.09637915 -0.5133328   0.23036276  0.13671806 -0.44083214
  0.03167189  0.34657738 -0.2259808   0.8407482  -0.27549633 -0.01389752
 -0.26075292  0.1958615   0.13791148 -0.01906174  0.17548484  0.20360112
 -0.5210762   0.29568496 -0.01811675 -0.08885522 -0.40023905  0.2212009
  0.3894793   0.18531346  0.20948155 -0.1227252  -0.34855083  0.390476
 -0.3892162   0.39152884 -0.38797048  0.13615257 -0.01593587  0.07370895
  0.2081034  -0.0653589  -0.10732474  0.02825111  0.34691438 -0.01557314
  0.17172274  0.02897884  0.45616502  0.07523499  0.01020092 -0.2440379
  0.10094997 -0.18405356 -0.06563722 -0.03816706  0.2340

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [21]:
print(model['fat-free cheddar cheese'])

[ 0.02765419 -0.3294496  -0.11896234  0.07646666 -0.03177109 -0.19033597
  0.1781223  -0.10507465 -0.22369273  0.01017765  0.00442666 -0.18320979
  0.09179017  0.19890773 -0.14012131  0.08493737 -0.13108899 -0.29785502
  0.08763244 -0.09410854  0.1476967   0.02608843 -0.19978258 -0.18477829
  0.06109774  0.05776844  0.02698181 -0.04919937  0.0893195  -0.07810136
  0.10709336 -0.07905519 -0.3089982   0.03402254  0.04226885 -0.02037675
 -0.02047033  0.18403427  0.00157852  0.23123506 -0.2053482  -0.11461245
 -0.06287142  0.17571093  0.01723172 -0.17312978  0.01038291  0.09462248
 -0.2331819   0.16259928 -0.10425939 -0.06645498 -0.24252321  0.14487211
  0.19511418  0.04140241  0.14445166 -0.09119292 -0.1851803   0.17326014
 -0.2767162   0.1031826  -0.04636717  0.17726503 -0.07673072 -0.0374202
 -0.06721064 -0.33100614 -0.14314881  0.05657972  0.05662482  0.11688674
  0.12864712  0.02266815  0.03844867 -0.08268416  0.10223137 -0.00997653
 -0.03915552  0.02917193 -0.08159602 -0.02842142 -0.

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [22]:
model['fat-free cheddar cheese'].shape

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(100,)

In [23]:
type(model['fat-free cheddar cheese'])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [34]:
import time
import pandas as pd
import numpy as np
def getVector_v2(cutWords, model):
        vector_list = [ model[k] for k in cutWords if k in model]
        vector_df = pd.DataFrame(vector_list)
        cutWord_vector = vector_df.mean(axis=0).values
        return cutWord_vector
 
startTime = time.time()
vector_list = []
i = 0
for cutWords in train['ingredients'][:]:
        i += 1
        if i % 1000 ==0:
                print('前%d篇文章形成词向量花费%.2f秒' %(i, time.time()-startTime))
 
        vector_list.append( getVector_v2(cutWords, model) )
X = np.array(vector_list)
print('Total Time You Need To Get X:%.2f秒' % (time.time() - startTime) )

D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


前1000篇文章形成词向量花费5.07秒
前2000篇文章形成词向量花费10.04秒
前3000篇文章形成词向量花费14.94秒
前4000篇文章形成词向量花费20.09秒
前5000篇文章形成词向量花费25.12秒
前6000篇文章形成词向量花费30.02秒
前7000篇文章形成词向量花费34.99秒
前8000篇文章形成词向量花费39.93秒
前9000篇文章形成词向量花费45.04秒
前10000篇文章形成词向量花费49.95秒
前11000篇文章形成词向量花费54.85秒
前12000篇文章形成词向量花费59.75秒
前13000篇文章形成词向量花费64.62秒
前14000篇文章形成词向量花费69.53秒
前15000篇文章形成词向量花费74.43秒
前16000篇文章形成词向量花费79.58秒
前17000篇文章形成词向量花费84.52秒
前18000篇文章形成词向量花费89.40秒
前19000篇文章形成词向量花费94.30秒
前20000篇文章形成词向量花费99.22秒
前21000篇文章形成词向量花费104.11秒
前22000篇文章形成词向量花费109.06秒
前23000篇文章形成词向量花费113.93秒
前24000篇文章形成词向量花费118.82秒
前25000篇文章形成词向量花费123.69秒
前26000篇文章形成词向量花费128.58秒
前27000篇文章形成词向量花费133.44秒
前28000篇文章形成词向量花费138.47秒
前29000篇文章形成词向量花费143.36秒
前30000篇文章形成词向量花费148.25秒
前31000篇文章形成词向量花费153.12秒
前32000篇文章形成词向量花费158.03秒
前33000篇文章形成词向量花费162.91秒
前34000篇文章形成词向量花费167.81秒
前35000篇文章形成词向量花费172.72秒
前36000篇文章形成词向量花费177.60秒
前37000篇文章形成词向量花费182.74秒
前38000篇文章形成词向量花费187.62秒
前39000篇文章形成词向量花费192.60秒
Total Time You Need To Get X:196.42秒


In [35]:
X

array([[ 0.00488153, -0.07471909, -0.00694176, ..., -0.09361217,
        -0.00357188, -0.20820458],
       [-0.01650314, -0.22604176, -0.18595559, ..., -0.14740083,
         0.09917212, -0.07850694],
       [-0.03483694, -0.25671606, -0.15828884, ..., -0.27964889,
         0.20572102, -0.07088326],
       ...,
       [-0.01280788, -0.28044556, -0.06630122, ..., -0.02941976,
         0.15684431,  0.08953508],
       [ 0.04640385, -0.1859432 , -0.00620176, ..., -0.21739457,
         0.13675954,  0.23046296],
       [-0.02757431, -0.33488105, -0.08490062, ..., -0.22442741,
         0.06255712,  0.09069215]])

In [36]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import cross_val_score

In [37]:
train['ingredients'][0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [38]:
model['romaine lettuce']

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.09070955,  0.1133802 , -0.16740635,  0.1941014 , -0.3537617 ,
       -0.38989943,  0.34018862, -0.30592024, -0.13780987,  0.20218177,
        0.251847  , -0.28722754,  0.25019342,  0.00870686,  0.0883423 ,
        0.13930233, -0.13595046, -0.28857714, -0.08695273,  0.05164268,
        0.25147578,  0.3314705 , -0.14620464, -0.2820956 ,  0.36720505,
        0.6133429 , -0.00522876, -0.3871687 ,  0.02790113, -0.18547904,
        0.13135074,  0.29544273, -0.02498611,  0.45833763, -0.30237803,
        0.20047826,  0.07082342,  0.09699158,  0.59738684,  0.15671134,
       -0.14030814, -0.20544128, -0.36753574,  0.24393508, -0.06155745,
       -0.26306668,  0.22393095,  0.06175306, -0.27223343, -0.06203232,
       -0.22599368,  0.07416092, -0.5458875 ,  0.41954687,  0.5567233 ,
       -0.21889465,  0.39541775, -0.366057  , -0.20845531,  0.03790593,
       -0.31425902,  0.12817743,  0.30297586,  0.09754949,  0.09620361,
        0.09419651, -0.17189068, -0.23790205, -0.5074029 , -0.16

In [39]:
X[1]

array([-0.01650314, -0.22604176, -0.18595559,  0.0180512 , -0.21374847,
        0.02562933, -0.10925157, -0.01711627, -0.07635584,  0.03027165,
       -0.08693466, -0.04535807,  0.15112986,  0.23489961, -0.28358136,
       -0.10966772, -0.08942102, -0.44438154, -0.0988734 ,  0.15381991,
        0.11669415,  0.15823055, -0.04814266, -0.06999438,  0.02483705,
       -0.04523937, -0.05516823, -0.00991642,  0.27292706, -0.03761397,
        0.19777931, -0.16810852,  0.02254523, -0.04476321,  0.13285588,
       -0.13745524, -0.00244369,  0.30701678, -0.10558816,  0.00224637,
        0.14185293, -0.18548527, -0.31875051,  0.14336071, -0.06292271,
        0.19101338,  0.17490362,  0.00055844, -0.05386827,  0.22104462,
        0.13277481, -0.09514338, -0.10498018,  0.05686245, -0.0507522 ,
       -0.01311225, -0.20479924, -0.14142071,  0.1873722 ,  0.06127356,
       -0.28509108,  0.18303498,  0.09521585,  0.08317706, -0.18013829,
        0.01594441, -0.06245564, -0.05744828, -0.18888618,  0.17

In [40]:
X[1800]

array([ 0.00121206, -0.16203284,  0.16524503, -0.1716917 , -0.33203769,
       -0.25363306,  0.02139137, -0.00932025, -0.14737728, -0.0101067 ,
        0.31648949,  0.07612023,  0.11923897,  0.21287747,  0.00306039,
        0.03202882,  0.01518856, -0.3215014 , -0.21022508, -0.06933635,
        0.09400438,  0.27209897, -0.17149662, -0.1887246 ,  0.05130234,
        0.26942399, -0.15383063, -0.37897875,  0.063091  ,  0.09594513,
        0.3734941 , -0.07536975, -0.25382844, -0.02212497, -0.27093068,
       -0.0555742 ,  0.14068938,  0.20269871, -0.10992669,  0.12935139,
        0.04528682, -0.03906621, -0.08384427,  0.09535031, -0.01352208,
        0.07907843,  0.15294936,  0.06706166, -0.06240052,  0.06309771,
       -0.11751459, -0.29678931, -0.2777422 ,  0.19340875,  0.12738796,
       -0.31954821, -0.03724808, -0.07249072,  0.07687027,  0.03555681,
       -0.2790867 ,  0.2224373 ,  0.23381145,  0.00530235, -0.10286504,
       -0.24789964,  0.08216632,  0.13244858, -0.08625729,  0.03

In [42]:
classifier = MultinomialNB()
classifier.fit = (train_x,train_y)

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, train['label'], test_size=0.3, random_state=0)
 
logistic_model = LogisticRegression()
logistic_model.fit(train_X, train_y)
logistic_model.score(test_X, test_y)

D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7265566077264728

In [52]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import cross_val_score
NB_model = GaussianNB()
NB_model.fit(train_X, train_y)
NB_model.score(test_X, test_y)

0.5730327662783876

In [60]:
X[1].shape

(100,)

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
param = {"n_neighbors": [3,5,10]}
gc = GridSearchCV(knn_model,param_grid=param ,cv=10)
gc.fit(train_X, train_y)
print("在测试集上的准确率：",gc.score(test_X, test_y))
print("选择最好的模型是：",gc.best_estimator_)
print("每个超参数每次交叉验证的结果：",gc.cv_results_)

在测试集上的准确率： 0.7404676108271181
选择最好的模型是： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.20005236, 0.20235014, 0.19914343]), 'std_fit_time': array([0.00230057, 0.01033799, 0.00354395]), 'mean_score_time': array([14.00058517, 14.52305827, 15.04352744]), 'std_score_time': array([0.23395227, 0.24456226, 0.09648512]), 'param_n_neighbors': masked_array(data=[3, 5, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 10}], 'split0_test_score': array([0.71786609, 0.72932331, 0.73540995]), 'split1_test_score': array([0.68960573, 0.70752688, 0.71935484]), 'split2_test_score': array([0.70501792, 0.72114695, 0.72939068]), 'split3_test_score': array([0.71367061, 0.7294582 , 0.73448152]), 'split4_test_score': ar

In [80]:
from sklearn.ensemble import RandomForestClassifier
param = {"n_estimators":[120,200,300,500,800,1200],"max_depth":[5,8,15,25,30]}
rf = RandomForestClassifier()
gc = GridSearchCV(rf,param_grid=param,cv=2)
gc.fit(train_X, train_y)
print("预测准确率：",gc.score(test_X, test_y))
print("查看选择的参数模型：",gc.best_params_)

预测准确率： 0.7362775496522249
查看选择的参数模型： {'max_depth': 30, 'n_estimators': 1200}


In [86]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [87]:
svc_model = LinearSVC()
svc_model.fit(train_X, train_y)
svc_model.score(test_X, test_y)

0.725215788150507

In [116]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
param = {"C":[0.1, 1, 10]}
svc_model = LinearSVC()
gc = GridSearchCV(svc_model,param_grid=param,cv=10)
gc.fit(train_X, train_y)
print("预测准确率：",gc.score(test_X, test_y))
print("查看选择的参数模型：",gc.best_params_)

D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\bas

预测准确率： 0.7252995893740048
查看选择的参数模型： {'C': 10}


D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
